## Dependencies

In [97]:
!pip install clean-text

In [98]:
import numpy as np
import pandas as pd
import re
from cleantext import clean
import json
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\omara\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\omara\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\omara\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Parsing JSON File

In [99]:
#! load data given file path and number of line to load
def load_data(file_path,num_lines):
    data = []
    with open(file_path, 'r') as file: 
        for _ in range(num_lines):
            data.append(json.loads(file.readline())) 
    return data

In [100]:
data=load_data('PIZZA_train.json',1000)
print(data[:5])

[{'train.SRC': 'four pizzas with extra american cheese and i want five lunch sized pizzas no american cheese', 'train.EXR': '(ORDER (PIZZAORDER (NUMBER 5 ) (SIZE LUNCH_SIZE ) (NOT (TOPPING AMERICAN_CHEESE ) ) ) (PIZZAORDER (NUMBER 4 ) (COMPLEX_TOPPING (QUANTITY EXTRA ) (TOPPING AMERICAN_CHEESE ) ) ) )', 'train.TOP': '(ORDER (PIZZAORDER (NUMBER four ) pizzas with (COMPLEX_TOPPING (QUANTITY extra ) (TOPPING american cheese ) ) ) and i want (PIZZAORDER (NUMBER five ) (SIZE lunch sized ) pizzas no (NOT (TOPPING american cheese ) ) ) )', 'train.TOP-DECOUPLED': '(ORDER (PIZZAORDER (NUMBER four ) (COMPLEX_TOPPING (QUANTITY extra ) (TOPPING american cheese ) ) ) (PIZZAORDER (NUMBER five ) (SIZE lunch sized ) (NOT (TOPPING american cheese ) ) ) )'}, {'train.SRC': 'can i have a party - sized pie without any bean', 'train.EXR': '(ORDER (PIZZAORDER (NUMBER 1 ) (SIZE PARTY_SIZE ) (NOT (TOPPING BEANS ) ) ) )', 'train.TOP': '(ORDER can i have (PIZZAORDER (NUMBER a ) (SIZE party - sized ) pie without 

In [101]:
#! split json data
def get_training_data(data):
    training_data = []
    training_exr=[]
    training_top=[]
    training_top_dec=[]
    for item in data:
        training_data.append(item['train.SRC'])
        training_exr.append(item['train.EXR'])
        training_top.append(item['train.TOP'])
        training_top_dec.append(item['train.TOP-DECOUPLED'])
    return training_data,training_exr,training_top,training_top_dec
training_data,training_exr,training_top,training_top_dec=get_training_data(data)
print(training_data[:2])
print(training_exr[:2])
print(training_top[:2])
print(training_top_dec[:2])

['four pizzas with extra american cheese and i want five lunch sized pizzas no american cheese', 'can i have a party - sized pie without any bean']
['(ORDER (PIZZAORDER (NUMBER 5 ) (SIZE LUNCH_SIZE ) (NOT (TOPPING AMERICAN_CHEESE ) ) ) (PIZZAORDER (NUMBER 4 ) (COMPLEX_TOPPING (QUANTITY EXTRA ) (TOPPING AMERICAN_CHEESE ) ) ) )', '(ORDER (PIZZAORDER (NUMBER 1 ) (SIZE PARTY_SIZE ) (NOT (TOPPING BEANS ) ) ) )']
['(ORDER (PIZZAORDER (NUMBER four ) pizzas with (COMPLEX_TOPPING (QUANTITY extra ) (TOPPING american cheese ) ) ) and i want (PIZZAORDER (NUMBER five ) (SIZE lunch sized ) pizzas no (NOT (TOPPING american cheese ) ) ) )', '(ORDER can i have (PIZZAORDER (NUMBER a ) (SIZE party - sized ) pie without any (NOT (TOPPING bean ) ) ) )']
['(ORDER (PIZZAORDER (NUMBER four ) (COMPLEX_TOPPING (QUANTITY extra ) (TOPPING american cheese ) ) ) (PIZZAORDER (NUMBER five ) (SIZE lunch sized ) (NOT (TOPPING american cheese ) ) ) )', '(ORDER (PIZZAORDER (NUMBER a ) (SIZE party - sized ) (NOT (TOPPING 

In [102]:
#! a function to handle negations
def handle_negations(text):
    #! negations are based on training set
    negations_pattern = r"\b(?:no|not|without)\s+.*?\b(?=(?:[^\w\s]|$))"
    # print(re.findall(negations_pattern, text))
    text = re.sub(negations_pattern, lambda x: ' '.join([f'not_{word}' for word in x.group(0).split()]), text)
    return text

In [103]:
#!# lemmatize words with all possible pos tags
def lem_word(word):
    possible_pos = [wordnet.NOUN, wordnet.VERB, wordnet.ADJ, wordnet.ADV]
    lemmatizer = WordNetLemmatizer()
    for pos in possible_pos:
        word=lemmatizer.lemmatize(word,pos)
    return word


In [104]:
lemmatizer = WordNetLemmatizer()
#! stopwords list (adding not_)
stopwords = set(stopwords.words('english'))
stopwords.add('like')
not_stopwords = ['not_' + word for word in stopwords]
stopwords.update(not_stopwords)
stopwords.discard('a')
stopwords.discard('an')
stopwords.discard('not')
stopwords.discard('no')
stopwords.discard('can')
stopwords.discard('not_a')
stopwords.discard('not_an')
stopwords.discard('not_can')
stopwords.discard('not_no')
# stopwords=set()
# # stopwords.add('and')
# # stopwords.add('also')

In [105]:
#! tokenize training data and remove stop words
def preprocess_training_data(training_data, stopwords):
    # training_data = [handle_negations(order) for order in training_data]
    training_data = [word_tokenize(order) for order in training_data]
    training_data = [[word.lower() for word in order if word.lower() not in stopwords] for order in training_data]
    training_data = [clean(order, no_line_breaks=True, no_punct=True, no_currency_symbols=True) for order in training_data]
    #! remove d letter most probably garbage
    training_data = [re.sub(r'\bd\s+', '', order) for order in training_data]
    #! remove "can" at the beginning of the sentence
    # training_data=[re.sub(r'^can\s+', '', order) for order in training_data]
    training_data = [word_tokenize(order) for order in training_data]
    # training_data = [[lemmatizer.lemmatize(word) for word in order] for order in training_data]
    training_data = [[lem_word(word) for word in order] for order in training_data]
    return training_data
training_data = preprocess_training_data(training_data, stopwords)
print(training_data[:5])

[['four', 'pizza', 'with', 'extra', 'american', 'cheese', 'and', 'i', 'want', 'five', 'lunch', 'size', 'pizza', 'no', 'american', 'cheese'], ['can', 'i', 'have', 'a', 'party', 'size', 'pie', 'without', 'any', 'bean'], ['can', 'i', 'have', 'a', 'large', 'bbq', 'pull', 'pork', 'can', 'i', 'have', 'a', 'large', 'bbq', 'pull', 'pork', 'can', 'i', 'have', 'a', 'large', 'bbq', 'pull', 'pork'], ['i', 'like', '7', 'pizza', 'with', 'bacon', 'and', 'eight', 'pizza', 'with', 'white', 'onion'], ['three', 'pizza', 'no', 'american', 'cheese', 'and', 'two', 'extra', 'large', 'pie', 'with', 'a', 'little', 'bite', 'of', 'bean']]


In [106]:
#! save processed training dataset
with open('training_data_processed.txt', 'w') as f:
    for item in training_data:
        f.write("%s\n" % item)

In [107]:
#! a utility function for extra parentheses ) removal 
#! handles COMPLEX_TOPPING, NOT,... parenthesis cases
def remove_unmatched_parentheses(input_string):
    result = list(input_string)  # Convert to list for mutability
    last_bracket_index=-1
    for i, char in enumerate(result):
        if char == ')' and i+2 < len(result):
            result[i] = ''  
            last_bracket_index=i
        elif char == '(':
            if last_bracket_index!=-1:
                result[last_bracket_index] = ')'
                last_bracket_index=-1
    return ''.join(result)

In [108]:
#! get PIZZAORDER, DRINKORDER, NONE Labels 0=>PIZZAORDER, 1=>DRINKORDER, 2=>NONE

def get_order_category_labels(training_top, training_data, stopwords):
    order_category_labels = []
    for i, item in enumerate(training_top):
        order_category_labels.append([2] * len(training_data[i]))
        unwanted_keywords = r"\b(ORDER|SIZE|STYLE|TOPPING|COMPLEX_TOPPING|QUANTITY|NOT|NUMBER|DRINKTYPE|CONTAINERTYPE|VOLUME)\b"
        cleaned_string = re.sub('\('+unwanted_keywords, "", item)
        cleaned_string = [word for word in cleaned_string.split() if word.lower() not in stopwords]
        cleaned_string = ' '.join(cleaned_string)
        cleaned_string = remove_unmatched_parentheses(cleaned_string)
        order_regex = r"\((?:PIZZAORDER|DRINKORDER).*?\)"
        extracted_orders = re.findall(order_regex, cleaned_string)
        k = 0
        for order in extracted_orders:
            order = re.sub(r"[\(\)]", "", order)
            order=clean(order, no_line_breaks=True, no_punct=True, no_currency_symbols=True)
            tokens = word_tokenize(order)
            # tokens = [lemmatizer.lemmatize(word) for word in tokens]
            tokens = [lem_word(word) for word in tokens]
            j = 0
            to_index_train=training_data[i][k:]
            if 'pizzaorder' in tokens:
                tokens.remove('pizzaorder')
                for word in to_index_train:
                    if j == len(tokens):
                        break
                    if word == tokens[j]:
                        order_category_labels[i][k] = 0
                        j += 1
                    k += 1
            elif 'drinkorder' in tokens:
                tokens.remove('drinkorder')
                for word in to_index_train:
                    if j == len(tokens):
                        break
                    if word == tokens[j]:
                        order_category_labels[i][k] = 1
                        j += 1
                    k += 1
    return order_category_labels

order_category_labels = get_order_category_labels(training_top, training_data, stopwords)
print(order_category_labels[:5])
    

[[0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0], [2, 2, 2, 0, 0, 0, 0, 0, 0, 0], [2, 2, 2, 0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 2, 2, 2, 1, 1, 1, 1, 1], [2, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [109]:
for labels in order_category_labels:
    with open('order_category_labels.txt', 'a') as f:
        f.write("%s\n" % labels)